Az osztály(oka)t mentsd a __src/linear_regression__ modul __LinearRegressions.py__ fájljába
Használható modulok: _pathlib, pandas, typing, str, numpy_, valamint a _scipy.stats t_ és _f_ osztályai

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path


In [13]:
import pandas as pd
import numpy as np
from scipy.stats import t
from scipy.stats import f

In [ ]:
datalib = Path.cwd().parent.joinpath('data')

In [4]:
sp500 = pd.read_parquet('/Users/feketedalma/Documents/GitHub/ECOPY_23241/data/sp500.parquet', engine = 'fastparquet')

In [5]:
factors = pd.read_parquet('/Users/feketedalma/Documents/GitHub/ECOPY_23241/data/ff_factors.parquet', engine = 'fastparquet')

In [6]:
merged_df = pd.merge(sp500, factors, on='Date', how='left')

In [7]:
merged_df['Excess Return'] = merged_df['Monthly Returns'] - merged_df['RF']

In [8]:
merged_df = merged_df.sort_values(by='Date')
merged_df['ex_ret_1'] = merged_df.groupby('Symbol')['Excess Return'].shift(-1)

In [9]:
merged_df = merged_df.dropna(subset=['ex_ret_1'])
merged_df = merged_df.dropna(subset=['HML'])

### Adatelőkészítés (0 pont)
1., Olvasd be a data mappa __sp500.parquet__ nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
2., Olvasd be az __ff_factors.parquet__ fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a __'Date' elsődleges kulcs__ alapján.
4., Készíts egy új __'Excess Return'__ nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
5., <u>Rendezd sorba dátum szerint az adatokat</u>, majd generálj egy új oszlopot (__'ex_ret_1'__), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a <u>következő időszaki Excess Return</u> érték. 
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer <u>törlöd az össze olyan sort</u>, amely az __'ex_ret_1' oszlopban hiányos__, majd ezt követően, törlöd az összes olyan sort, ami a __'HML' oszlopban hiányos__.


3. feladat segítség
![Joined data](../resources/weekly6/joined_data.jpg)

5. feladat segítség
![new column](../resources/weekly6/ex_ret_1.jpg)

### Modell összeállítás (7 pont)

A meglévő adatokból válaszd ki a __Amazon részvényhez tartozó sorokat (AMZN)__ és töröld a tickereket tartalmazó oszlopot.
 <u>Ebben a blokkban önellenőrzésre használhatod a legutóbbi zárthelyin írt osztályodat.</u>

7., Készíts egy új __LinearRegressionNP__ elnevezésű osztályt. Definiáld benne a __\_\_init\_\___ nevű függvényt, amely bemenetként 2 DataFrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (__left_hand_side__), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (__right_hand_side__).

8., Egésztsd ki az osztályt egy __fit__ metódussal, ami OLS elvű becslést hajt végre. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között. <u>__(numpy.linalg.lstsq() nem használható)__</u>

9., Egészítsd ki az osztályt egy __get_params__ metódussal, ami visszaadja a becsült modell béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen __Beta coefficients__. 

10., Egészítsd ki az osztályt egy __get_pvalues__ metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: __P-values for the corresponding coefficients__. A p értéket t-statisztika alapján számold ki. A p-érték kiszámításánál figyelj alkalmazd a <u>min(value, 1-value) * 2</u> képletet.

11., Egészítsd ki az osztályt egy __get_wald_test_result__ metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Wald: wald_value, p-value: p_value__, ahol az wald_value és p_value helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. A függvény bemenete során feltételezzük, hogy r minden eleme 0, R-t listák listája formában adjuk át. A Wald statisztika p értékét az előző feladathoz hasonlóan számold ki, de figyelj, hogy a <u>teszt 1 oldalú</u>.

12., Egészítse ki az osztályt egy __get_model_goodness_values__ metódussal, ami visszadja a centrált és a módosított R-négyzet értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Centered R-squared: crs, Adjusted R-squared: ars__, ahol crs és ars helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. <u>Ha a regresszorok számába eredetileg beleszámítottad a konstanst is, akkor a módosított R-négyzet számítás nevezőjében nincs szükség a __-1__-es tagra</u>.

In [10]:
amazon = merged_df[merged_df['Symbol'] == 'AMZN']
amazon = amazon.drop(columns=['Symbol'])

In [ ]:
class LinearRegressionNP:
    def __init__(self, left_hand_side, right_hand_side):
        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self.model = None

    def fit(self):
        X = self.right_hand_side
        y = self.left_hand_side
        X = np.column_stack((np.ones(X.shape[0]), X))
        XTX = np.dot(X.T, X)
        XTy = np.dot(X.T, y)
        beta = np.linalg.solve(XTX, XTy)
        alpha = beta[0]
        beta = beta[1:]
        return alpha, beta 

    def get_params(self):
        X = self.right_hand_side
        y = self.left_hand_side
        X = np.column_stack((np.ones(X.shape[0]), X))

        XTX = np.dot(X.T, X)
        XTy = np.dot(X.T, y)

        beta = np.linalg.solve(XTX, XTy)
        beta_series = pd.Series(beta[1:], name="Beta coefficients")
        return beta_series

    def get_pvalues(self):
        X = self.right_hand_side
        y = self.left_hand_side
        X = np.column_stack((np.ones(X.shape[0]), X))

        XTX = np.dot(X.T, X)
        XTy = np.dot(X.T, y)

        beta = np.linalg.solve(XTX, XTy)
        y_pred = np.dot(X, beta)
        residuals = y - y_pred
        mse = np.sum(residuals**2) / (X.shape[0] - X.shape[1])
        t_stats = beta / np.sqrt(np.diagonal(mse * np.linalg.inv(XTX)))
        p_values = 2 * (1 - t.cdf(np.abs(t_stats), X.shape[0] - X.shape[1]))
        p_values_series = pd.Series(p_values[1:], name="P-values for the corresponding coefficients")
    
        return p_values_series

    def get_wald_test_result(self, R):
    
        X = self.right_hand_side
        y = self.left_hand_side
        X = np.column_stack((np.ones(X.shape[0]), X))

        XTX = np.dot(X.T, X)
        XTy = np.dot(X.T, y)
        beta_hat = np.linalg.solve(XTX, XTy)

        n = X.shape[0]
        residuals = y - np.dot(X, beta_hat)
        sse = np.sum(residuals**2)

        wald_value = np.dot(np.dot(R, beta_hat), R.T) / sse

        df1 = R.shape[0]
        df2 = n - X.shape[1]

        p_value = 1 - f.cdf(wald_value, df1, df2)

        result_string = f"Wald: {wald_value:.3f}, p-value: {p_value:.3f}"
    
        return result_string

    def get_model_goodness_values(self):
        
        X = self.right_hand_side
        y = self.left_hand_side
        n = X.shape[0]
        p = X.shape[1] 

        X = np.column_stack((np.ones(n), X))

    # Az OLS becsléshez szükséges mátrixok kiszámítása
        XTX = np.dot(X.T, X)
        XTy = np.dot(X.T, y)

    # A modell hibakvadrátumának számítása
        residuals = y - np.dot(X, self.get_params())
        sse = np.sum(residuals**2)

    # A teljes variabilitás számítása
        sst = np.sum((y - np.mean(y))**2)

    # Centrált R-négyzet számítása
        crs = 1 - sse / sst

    # Módosított R-négyzet számítása
        ars = 1 - (sse / (n - p - 1)) / (sst / (n - 1))

    # Visszaadjuk az eredményt a megadott formátumban
        result_string = f"Centered R-squared: {crs:.3f}, Adjusted R-squared: {ars:.3f}"

        return result_string